In [85]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [86]:
# Read the csv file into pandas Dataframe
credit_data = pd.read_csv('Resources/credit_data.csv')
credit_data.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [87]:
## Basic formatting for ease of use/prevent errors

# Lowercase for all headings
credit_data.columns = credit_data.columns.str.lower()
credit_data.columns

Index(['id', 'customer_id', 'month', 'name', 'age', 'ssn', 'occupation',
       'annual_income', 'monthly_inhand_salary', 'num_bank_accounts',
       'num_credit_card', 'interest_rate', 'num_of_loan', 'type_of_loan',
       'delay_from_due_date', 'num_of_delayed_payment', 'changed_credit_limit',
       'num_credit_inquiries', 'credit_mix', 'outstanding_debt',
       'credit_utilization_ratio', 'credit_history_age',
       'payment_of_min_amount', 'total_emi_per_month',
       'amount_invested_monthly', 'payment_behaviour', 'monthly_balance',
       'credit_score'],
      dtype='object')

In [ ]:
## Get an understanding the data
# credit_data.info()

In [ ]:
## Drop columns that aren't necessary

# 'ID' is persons ID, arbitrary, not predictive in credit scoring, not required
# 'Customer_ID' as above
# 'Name' as above
# 'SSN' as above

credit_data_1 = credit_data.drop(columns=["id", "customer_id", "name", "ssn"])
credit_data_1.head()

### Preprocessing

##### Working through each columns to understand anomalous values, and alter or delete as appropriate

In [89]:
# Identify duplicates by SSN 

credit_data_1[credit_data_1.duplicated(['id'])]

KeyError: Index(['id'], dtype='object')

In [ ]:
## AGE

# Understand the range of ages, and if age values make sense
credit_data_1["age"].value_counts().sort_index().head(20)

# From perusal of column data, can see many values include'_'.
# Count how many rows with an underscore
US_credit_data_1 = credit_data_1["age"].str.contains('_').sum()
print(f"{US_credit_data_1} out of 100k rows, have a string, 4%")

# Remove underscore '_'
# credit_data_1.loc[credit_data_1["age"] == "_", "age"] = ""
    # code for where value is '_' only, not part of value is '_'
# Code to replace '_' with ""
credit_data_1["age"] = credit_data_1["age"].str.replace("_", "")

# Try changing all values to numeric. Will prove no more anomalous values
credit_data_1["age"] = pd.to_numeric(credit_data_1["age"])
f"Able to convert all values to numeric, suggesting, no more anomalous values"


In [ ]:
## Many ages > 85 and <14 (assuming 14 is universal age ppl are allowed to be officially employed)
# Count how many values are <18 and >90, as unlikely to be seeking loans.
# If insignificant, can delete rows
credit_data_2 = credit_data_1.loc[(credit_data_1["age"] < 14) | (credit_data_1["age"] > 85)]
print(credit_data_2["age"].describe())
length_18_85 = len(credit_data_2["age"])
f"{length_18_85} of 100k values are <14 & >85. Will be deleted"

# DF updated to exclude <14 and >85
credit_data_3 = credit_data_1.loc[(credit_data_1["age"] > 14) & (credit_data_1["age"] < 85)]


In [ ]:
## MONTH

credit_data_3["month"].value_counts()
# Data included from Jan - July.
# No anomalous entries.
# Data OK

In [ ]:
## OCCUPATION

credit_data_3["occupation"].value_counts()

# credit_data_1["occupation"].value_counts().sum()
# 7% of ppl don't have a recorded occupation.

# Will either convert "_______" (as copied from results below) Other.
credit_data_3.loc[credit_data_3["occupation"] == "_______", "occupation"] = "Other"
# credit_data_3

In [ ]:
## ANNUAL INCOME

credit_data_3["annual_income"].value_counts()
# There are strings of '_' in income. Need to delete, use code as above.
credit_data_3["annual_income"] = credit_data_3["annual_income"].str.replace("_", "")

# Convert to numeric
credit_data_3["annual_income"] = pd.to_numeric(credit_data_3["annual_income"])

# credit_data_3["annual_income"].describe()
# Count matches DF length, data OK.


In [ ]:
## MONTHLY INCOME

credit_data_3["monthly_inhand_salary"].value_counts()

# Convert to numeric
credit_data_3["monthly_inhand_salary"] = pd.to_numeric(credit_data_3["monthly_inhand_salary"])

# Data OK

In [ ]:
## NUM OF BANK ACCS

credit_data_3["num_bank_accounts"].value_counts()
# Shows values in the 1000s, intuitively wrong, unlikely for a person to have 1000 bank accounts.

# Count number of values with banks accounuts > than 20
bank_acc_20plus = credit_data_3.loc[(credit_data_3["num_bank_accounts"] > 20)]
print(bank_acc_20plus["num_bank_accounts"].describe())
f"1274 of 100k values, will be deleted"

credit_data_4 = credit_data_3.loc[(credit_data_3["num_bank_accounts"] < 20)]

# Convert to numeric
credit_data_4["num_bank_accounts"] = pd.to_numeric(credit_data_4["num_bank_accounts"])


In [ ]:
## NUM OF CREDIT CARDS

credit_data_4["num_credit_card"].value_counts()

# Shows values in the 100s, intuitively wrong, unlikely for a person to have 100s of credit cards.

# Count number of values with banks accounuts > than 10
cc_10plus = credit_data_4.loc[(credit_data_4["num_credit_card"] > 10)]
print(cc_10plus["num_credit_card"].describe())
f"2194 of 100k values, will be deleted"

credit_data_5 = credit_data_4.loc[(credit_data_4["num_credit_card"] <= 10)]

# Convert to numeric
credit_data_5["num_credit_card"] = pd.to_numeric(credit_data_5["num_credit_card"])

In [ ]:
## INTEREST_RATE

credit_data_5["interest_rate"].value_counts()

# Shows values in the 100s, unlikely that interest rates would be >50% on credit cards.

# Count number of values with banks accounuts > than 10
ir_30plus = credit_data_5.loc[(credit_data_5["interest_rate"] > 50)]
print(ir_30plus["interest_rate"].describe())
f"1899 of 100k values, will be deleted"

credit_data_6 = credit_data_5.loc[(credit_data_5["interest_rate"] <= 50)]

# Convert to numeric
credit_data_6["interest_rate"] = pd.to_numeric(credit_data_6["interest_rate"])

In [ ]:
## NUM_OF_LOAN

credit_data_6["num_of_loan"].value_counts()
credit_data_6["num_of_loan"].describe()
credit_data_6["num_of_loan"].min()
# Count number of values < 0
# loan_less0 = credit_data_6.loc[(credit_data_6["num_of_loan"] < 0)]
# Above code has strings in it. Assume its same string as before '_', delete this string.
credit_data_6["num_of_loan"] = credit_data_6["num_of_loan"].str.replace("_", "")

# Convert data to numeric
credit_data_6["num_of_loan"] = pd.to_numeric(credit_data_6["num_of_loan"])


In [ ]:
loan_less0 = credit_data_6.loc[(credit_data_6["num_of_loan"] < 0)]
# loan_less0["num_of_loan"].describe()
f"3512 of 100k values, will be deleted"

credit_data_7 = credit_data_6.loc[(credit_data_6["interest_rate"] >= 0)]

In [ ]:
## TYPE OF LOAN

credit_data_7["type_of_loan"].value_counts()
# Shows there are many values with nil, due to person having 0 loans.
# Replace blank values, with 'None'
latest = credit_data_7["type_of_loan"].fillna("None", inplace = True)



In [ ]:
## DELAY FROM DUE DATE

credit_data_7["delay_from_due_date"].value_counts()
# no anomalous figures. -ve values represent payments in advance.

In [ ]:
## NUM OF DELAYED PAYMENT

credit_data_7["num_of_delayed_payment"].value_counts()
credit_data_7["num_of_delayed_payment"].describe()

# From perusal, there are erroneous strings, remove
credit_data_7["num_of_delayed_payment"] = credit_data_7["num_of_delayed_payment"].str.replace("_", "")

# Replace blank values with 0 integer, to ability to calculate as a factor later
credit_data_7["num_of_delayed_payment"] = credit_data_7["num_of_delayed_payment"].fillna(0)

# Convert data to numeric
credit_data_7["num_of_delayed_payment"] = pd.to_numeric(credit_data_7["num_of_delayed_payment"])


In [ ]:
## CHANGED CREDIT LIMIT

credit_data_7["changed_credit_limit"].value_counts()
credit_data_7["changed_credit_limit"].describe()

# From perusal, there are erroneous strings, remove
credit_data_7["changed_credit_limit"] = credit_data_7["changed_credit_limit"].str.replace("_", "")

# Replace blank values with 0 integer, to ability to calculate as a factor later
credit_data_7["changed_credit_limit"] = credit_data_7["changed_credit_limit"].fillna(0)

In [ ]:
## NUM OF CREDIT INQUIRIES

credit_data_7["num_credit_inquiries"].value_counts()
credit_data_7["num_credit_inquiries"].describe()

# Replace blank values with 0 integer, to ability to calculate as a factor later
credit_data_7["num_credit_inquiries"] = credit_data_7["num_credit_inquiries"].fillna(0)

# Len same as DF whole, suggesting, all records (rows) have a value.

# Convert data to numeric
credit_data_7["num_credit_inquiries"] = pd.to_numeric(credit_data_7["num_credit_inquiries"])

In [ ]:
## CREDTI MIX

credit_data_7["credit_mix"].value_counts()
# credit_data_7["credit_mix"].describe()

# Replace blank values with 0 integer, to ability to calculate as a factor later
# credit_data_7["credit_mix"] = credit_data_7["credit_mix"].fillna(0)

# From value_counts, 1 category is '_', replace with 'None'
credit_data_7["credit_mix"] = credit_data_7["credit_mix"].str.replace("_", "None")

In [ ]:
## OUTSTANDING DEBT

credit_data_7["outstanding_debt"].value_counts()
# credit_data_7["credit_mix"].describe()

# From perusal, there are erroneous strings, remove
credit_data_7["outstanding_debt"] = credit_data_7["outstanding_debt"].str.replace("_", "")

# Convert column to numeric
credit_data_7["outstanding_debt"] = pd.to_numeric(credit_data_7["outstanding_debt"])


In [ ]:
## CREDIT UTILIZATION RATIO

credit_data_7["credit_utilization_ratio"].value_counts()

# Convert column to numeric
credit_data_7["credit_utilization_ratio"] = pd.to_numeric(credit_data_7["credit_utilization_ratio"])


In [ ]:
## CREDIT HISTORY AGE

credit_data_7["credit_history_age"].value_counts()

# Replace blank values with 0 integer, to ability to calculate as a factor later
credit_data_7["credit_history_age"] = credit_data_7["credit_history_age"].fillna("0")

In [ ]:
# Split credit_history_age into seperate columns, to perform: numeric, calcs and amalgamation, for later ML calcs
credit_data_7[["credit_history_age_year", "ystr", "andstr", "credit_history_age_month", "mstr"]] = credit_data_7.credit_history_age.str.split(" ", expand = True)

In [ ]:
# Drop redundant columns, 
credit_data_8 = credit_data_7.drop(columns=["credit_history_age", "ystr", "andstr", "mstr"], axis = 1)

In [ ]:
# Convert year and month to numeric
credit_data_8["credit_history_age_year"] = pd.to_numeric(credit_data_8["credit_history_age_year"])
credit_data_8["credit_history_age_month"] = pd.to_numeric(credit_data_8["credit_history_age_month"])


In [ ]:
# Convert month nominal to decimal (i.e /12)
months = credit_data_8["credit_history_age_month"]/12
credit_data_8["credit_history_age_mnthdec"] = months

In [ ]:
# Concactenate
history_age = credit_data_8["credit_history_age_year"] + credit_data_8["credit_history_age_mnthdec"]
credit_data_8["credit_history_age_new"] = history_age
# credit_data_8["credit_history_age_new"]


In [ ]:
# Drop redundant columns (split and decimals calcs)
credit_data_9 = credit_data_8.drop(columns=["credit_history_age_year", "credit_history_age_month", "credit_history_age_mnthdec"], axis = 1)
credit_data_9

In [ ]:
## PAYMENT OF MIN AMOUNT

credit_data_9["payment_of_min_amount"].value_counts()

# Convert year and month to numeric
credit_data_9["payment_of_min_amount"] = pd.to_numeric(credit_data_9["payment_of_min_amount"])

# Data OK

In [ ]:
## TOTAL EMI PER MONTH

credit_data_9["total_emi_per_month"].value_counts()

# Convert to numeric
credit_data_9["total_emi_per_month"] = pd.to_numeric(credit_data_9["total_emi_per_month"])

# Data OK

In [ ]:
## AMOUNT INVESTED MONTHLY

credit_data_9["amount_invested_monthly"].value_counts()

# Remove strings
credit_data_9["amount_invested_monthly"] = credit_data_9["amount_invested_monthly"].str.replace("_", "")

# Convert to numeric
credit_data_9["amount_invested_monthly"] = pd.to_numeric(credit_data_9["amount_invested_monthly"])
credit_data_9["amount_invested_monthly"]
# Data OK

In [ ]:
## MONTHLY BALANCE

credit_data_9["monthly_balance"].value_counts()

credit_data_9["monthly_balance"] = credit_data_9["monthly_balance"].str.replace("-333333333333333333333333333", "")
credit_data_9["monthly_balance"] = credit_data_9["monthly_balance"].str.replace("_", "")

# Pandas doesn't recognise empty string (i.e. blank cells). 
# Pandas will recognise a value as null if it is a np.nan.
# Covnert empty strings to np.nan. Then convert to 0
credit_data_9["monthly_balance"] = credit_data_9["monthly_balance"].replace('', np.nan)

# Replace NaN with 0
credit_data_9["monthly_balance"] = credit_data_9["monthly_balance"].fillna(0)

# Convert to numeric
credit_data_9["monthly_balance"] = pd.to_numeric(credit_data_9["monthly_balance"])


In [ ]:
## CREDIT SCORE

credit_data_9["credit_score"].value_counts()

In [ ]:
# Remove values < 0
loan_less0_1 = credit_data_9.loc[(credit_data_9["monthly_balance"] < 0)]
loan_less0_1["monthly_balance"].describe()
# f"3512 of 100k values, will be deleted"

# credit_data_7 = credit_data_6.loc[(credit_data_6["interest_rate"] >= 0)]